In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import interp1d

# === Input Parameters ===
concentration = 2  # g/L
molecular_weight = 8  # MDa
eta7_exp = 20  # Experimental viscosity at 7.3 s⁻¹, cP
temperature = 298  # K (25 °C)

# === Initial Viscosity Calculation ===
eta_infinite = np.exp(-3.7188 + (578.919 / (-137.546 + temperature)))

# === Estimate Intrinsic Viscosity ===
ceta = np.arange(0.1, 100.1, 0.1)
eta_0 = 1 + ceta + 0.582 * ceta**2.009 + 0.022 * ceta**4
term1 = -0.08212 * ceta
n = 1 - (0.6187 - 0.5203 * np.exp(term1))
term2 = (n - 1) / 2
lambda_d = 0.251 + 1.54 * molecular_weight * ceta / (concentration * temperature)
lambda_ = lambda_d * (0.810 + 0.0230 * ceta**2.438)
term3 = (lambda_ * 7.3)**2
eta7_model = 1 + (eta_0 - 1) * (1 + term3)**term2

# === Interpolate to Find Experimental ceta ===
interp_func = interp1d(eta7_model, ceta, fill_value="extrapolate")
ceta_exp = float(interp_func(eta7_exp))

# === Carreau-Yasuda Model Calculation ===
eta_0C = 1 + ceta_exp + 0.582 * ceta_exp**2.009 + 0.022 * ceta_exp**4
term1C = -0.08212 * ceta_exp
nC = 1 - (0.6187 - 0.5203 * np.exp(term1C))
term2C = (nC - 1) / 2
lambda_dC = 0.251 + 1.54 * molecular_weight * ceta_exp / (concentration * temperature)
lambdaC = lambda_dC * (0.810 + 0.0230 * ceta_exp**2.438)

shear_rate = np.arange(0.01, 1000.1, 0.1)
etaC = eta_infinite + (eta_0C - eta_infinite) * (1 + (lambdaC * shear_rate)**2)**term2C

# === Create DataFrame and Display Results ===
df = pd.DataFrame({
    "Shear Rate (1/s)": shear_rate,
    "Viscosity (cP)": etaC
})

print("First 10 rows of Carreau-Yasuda model output:")
print(df.head(10))

# === Plot using Plotly ===
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=shear_rate, y=etaC,
    mode='lines',
    name='T = 298 K'
))
fig.update_layout(
    title="Carreau-Yasuda Viscosity Curve",
    xaxis=dict(title='Shear Rate (1/s)', type='log'),
    yaxis=dict(title='Viscosity η (cP)', type='log'),
    template='plotly_white'
)
fig.show()

# === Optional: Save Data to TXT File ===
df.to_csv("carreau_data.txt", index=False, sep="\t")
print("File 'carreau_data.txt' successfully saved.")


First 10 rows of Carreau-Yasuda model output:
   Shear Rate (1/s)  Viscosity (cP)
0              0.01       29.701141
1              0.11       29.682776
2              0.21       29.634287
3              0.31       29.556952
4              0.41       29.452732
5              0.51       29.324119
6              0.61       29.173969
7              0.71       29.005324
8              0.81       28.821258
9              0.91       28.624746


File 'carreau_data.txt' successfully saved.
